# ETAPA 4: Análise das Variáveis de Escala Likert

## Consulta à Comunidade UFPE sobre o Uso de Inteligência Artificial

---

**Objetivo:** Avaliar grau de concordância com afirmações sobre IA.

**Variáveis analisadas (5 afirmações):**
1. "A IA já não é mais uma opção, é uma realidade"
2. "O uso ético requer diretrizes claras"
3. "A UFPE deve oferecer diretrizes sobre uso em avaliações"
4. "O uso de IA deve ser incentivado como ferramenta de apoio"
5. "A universidade deve promover debates sobre impactos da IA"

**Procedimentos:**
1. Calcular distribuição de frequências para cada nível de concordância
2. Calcular medidas de tendência central (mediana, moda)
3. Agrupar respostas em "Concordância", "Neutro" e "Discordância"
4. Estratificar por vínculo institucional e aplicar teste de Kruskal-Wallis
5. Gerar gráficos de barras empilhadas (stacked bar charts) para visualização

**Produto:** Tabelas de concordância; testes de diferença entre grupos; gráficos divergentes.

**Critério de Validação:** Consistência interna das respostas; significância estatística nas comparações entre grupos.

---

## Requisitos Técnicos

### Kernel Python
- **Python:** 3.9 ou superior
- **Kernel recomendado:** `python3` ou ambiente virtual com as dependências instaladas

### Dados de Entrada

| Arquivo | Descrição | Origem |
|---------|-----------|--------|
| `dados_limpos_etapa1.xlsx` | Base de dados limpa com 2.164 registros | Etapa 1 |

### Dados de Saída

| Arquivo | Descrição |
|---------|----------|
| `likert_etapa4.xlsx` | Tabelas de frequência, medidas descritivas e testes estatísticos |
| `grafico_likert_geral_etapa4.png` | Visão geral das 5 afirmações (barras divergentes) |
| `grafico_likert_detalhado_etapa4.png` | Distribuição detalhada por afirmação |
| `grafico_likert_vinculo_etapa4.png` | Comparação entre vínculos institucionais |
| `relatorio_likert_etapa4.md` | Relatório com resultados e testes estatísticos |

---

## 1. Instalação de Dependências

In [ ]:
# ============================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# Execute esta célula apenas uma vez para instalar os pacotes necessários
# ============================================================================

!pip install pandas openpyxl matplotlib seaborn numpy scipy --quiet

print("✅ Dependências instaladas com sucesso!")

## 2. Configuração Inicial

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import kruskal, mannwhitneyu
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
sns.set_style("whitegrid")

# Verificar versões
print("Bibliotecas carregadas com sucesso!")
print(f"  • Pandas: {pd.__version__}")
print(f"  • Scipy: {stats.__name__} (para testes estatísticos)")
print(f"  • Matplotlib: {plt.matplotlib.__version__}")
print(f"  • Seaborn: {sns.__version__}")

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DE ARQUIVOS DE ENTRADA E SAÍDA
# ============================================================================

# Arquivo de entrada (gerado na Etapa 1)
ARQUIVO_ENTRADA = "dados_limpos_etapa1.xlsx"

# Arquivos de saída
ARQUIVO_SAIDA_EXCEL = "likert_etapa4.xlsx"
ARQUIVO_SAIDA_RELATORIO = "relatorio_likert_etapa4.md"

# Nível de significância para testes estatísticos
ALPHA = 0.05

print("Configuração de arquivos:")
print(f"  📥 Entrada: {ARQUIVO_ENTRADA}")
print(f"  📤 Saídas:  {ARQUIVO_SAIDA_EXCEL}")
print(f"            {ARQUIVO_SAIDA_RELATORIO}")
print(f"  📊 Nível de significância (α): {ALPHA}")

## 3. Carregamento dos Dados

In [ ]:
# Carregar dados limpos da Etapa 1
try:
    df = pd.read_excel(ARQUIVO_ENTRADA)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   Registros: {len(df)}")
    print(f"   Variáveis: {len(df.columns)}")
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_ENTRADA}' não encontrado!")
    print(f"   Certifique-se de que a Etapa 1 foi executada e o arquivo está no diretório correto.")
    raise

In [ ]:
# ============================================================================
# IDENTIFICAÇÃO DAS VARIÁVEIS LIKERT
# ============================================================================

# Dicionário com as variáveis Likert e suas descrições curtas
variaveis_likert = {
    'Likert_IA_Realidade': 'IA é uma realidade com a qual precisamos lidar',
    'Likert_Diretrizes_Eticas': 'Uso ético requer diretrizes claras',
    'Likert_Diretrizes_Avaliacoes': 'UFPE deve oferecer diretrizes para avaliações',
    'Likert_Incentivo_IA': 'Uso de IA deve ser incentivado como apoio',
    'Likert_Debates_Impactos': 'Universidade deve promover debates sobre IA'
}

# Nomes curtos para os gráficos
nomes_curtos = {
    'Likert_IA_Realidade': 'IA é realidade',
    'Likert_Diretrizes_Eticas': 'Diretrizes éticas',
    'Likert_Diretrizes_Avaliacoes': 'Diretrizes avaliações',
    'Likert_Incentivo_IA': 'Incentivar IA',
    'Likert_Debates_Impactos': 'Promover debates'
}

print("=" * 70)
print("VARIÁVEIS DE ESCALA LIKERT PARA ANÁLISE")
print("=" * 70)

for var, descricao in variaveis_likert.items():
    print(f"\n📌 {var}")
    print(f"   Descrição: {descricao}")
    print(f"   Valores: {df[var].unique().tolist()}")

---

## 4. Definição da Escala e Funções Auxiliares

In [ ]:
# ============================================================================
# DEFINIÇÃO DA ESCALA LIKERT
# ============================================================================

# Ordem lógica da escala (de mais negativo para mais positivo)
ordem_likert = [
    'Discordo totalmente',
    'Discordo',
    'Neutro',
    'Concordo',
    'Concordo totalmente'
]

# Mapeamento numérico (para cálculos estatísticos)
mapa_numerico = {
    'Discordo totalmente': 1,
    'Discordo': 2,
    'Neutro': 3,
    'Concordo': 4,
    'Concordo totalmente': 5
}

# Mapeamento reverso
mapa_reverso = {v: k for k, v in mapa_numerico.items()}

# Agrupamento em 3 categorias
mapa_agrupado = {
    'Discordo totalmente': 'Discordância',
    'Discordo': 'Discordância',
    'Neutro': 'Neutro',
    'Concordo': 'Concordância',
    'Concordo totalmente': 'Concordância'
}

ordem_agrupado = ['Discordância', 'Neutro', 'Concordância']

# Cores para os gráficos (divergente: vermelho -> cinza -> verde)
cores_likert = {
    'Discordo totalmente': '#d73027',  # Vermelho escuro
    'Discordo': '#fc8d59',              # Vermelho claro
    'Neutro': '#ffffbf',                # Amarelo claro
    'Concordo': '#91cf60',              # Verde claro
    'Concordo totalmente': '#1a9850'    # Verde escuro
}

cores_agrupado = {
    'Discordância': '#e74c3c',
    'Neutro': '#f39c12',
    'Concordância': '#27ae60'
}

print("✅ Escala Likert configurada:")
print(f"   Níveis: {ordem_likert}")
print(f"   Agrupamento: {ordem_agrupado}")

In [ ]:
# ============================================================================
# FUNÇÕES AUXILIARES
# ============================================================================

def calcular_frequencias_likert(df, variavel):
    """
    Calcula frequências para uma variável Likert mantendo a ordem correta.
    """
    freq = df[variavel].value_counts().reindex(ordem_likert).fillna(0).astype(int)
    freq_df = pd.DataFrame({
        'Nível': freq.index,
        'Frequência': freq.values,
        'Percentual': (freq.values / freq.values.sum() * 100).round(1)
    })
    return freq_df


def calcular_medidas_centrais(df, variavel):
    """
    Calcula mediana, moda e média da variável Likert (convertida para numérica).
    """
    valores_numericos = df[variavel].map(mapa_numerico).dropna()
    
    mediana = valores_numericos.median()
    moda = valores_numericos.mode().values[0] if len(valores_numericos.mode()) > 0 else np.nan
    media = valores_numericos.mean()
    desvio = valores_numericos.std()
    
    return {
        'media': round(media, 2),
        'mediana': mediana,
        'mediana_texto': mapa_reverso.get(mediana, 'N/A'),
        'moda': moda,
        'moda_texto': mapa_reverso.get(moda, 'N/A'),
        'desvio_padrao': round(desvio, 2),
        'n': len(valores_numericos)
    }


def calcular_concordancia_agrupada(df, variavel):
    """
    Agrupa respostas em Concordância, Neutro e Discordância.
    """
    agrupado = df[variavel].map(mapa_agrupado)
    freq = agrupado.value_counts().reindex(ordem_agrupado).fillna(0).astype(int)
    freq_df = pd.DataFrame({
        'Categoria': freq.index,
        'Frequência': freq.values,
        'Percentual': (freq.values / freq.values.sum() * 100).round(1)
    })
    return freq_df


def teste_kruskal_wallis(df, variavel_likert, variavel_grupo='Vinculo_Padronizado'):
    """
    Realiza teste de Kruskal-Wallis para comparar grupos.
    Retorna estatística H, p-valor e interpretação.
    """
    # Converter para numérico
    df_temp = df[[variavel_likert, variavel_grupo]].dropna().copy()
    df_temp['valor_numerico'] = df_temp[variavel_likert].map(mapa_numerico)
    
    # Separar grupos
    grupos = [grupo['valor_numerico'].values for nome, grupo in df_temp.groupby(variavel_grupo)]
    nomes_grupos = list(df_temp[variavel_grupo].unique())
    
    # Realizar teste
    estatistica, p_valor = kruskal(*grupos)
    
    # Calcular tamanho do efeito (eta-quadrado)
    n_total = len(df_temp)
    k = len(grupos)
    eta_squared = (estatistica - k + 1) / (n_total - k)
    eta_squared = max(0, eta_squared)  # Garantir valor não negativo
    
    # Interpretação do tamanho do efeito
    if eta_squared < 0.01:
        efeito = "desprezível"
    elif eta_squared < 0.06:
        efeito = "pequeno"
    elif eta_squared < 0.14:
        efeito = "médio"
    else:
        efeito = "grande"
    
    return {
        'variavel': variavel_likert,
        'estatistica_H': round(estatistica, 3),
        'p_valor': p_valor,
        'eta_squared': round(eta_squared, 4),
        'efeito': efeito,
        'significativo': p_valor < ALPHA,
        'n_grupos': k,
        'grupos': nomes_grupos
    }


def calcular_medianas_por_grupo(df, variavel_likert, variavel_grupo='Vinculo_Padronizado'):
    """
    Calcula a mediana da variável Likert para cada grupo.
    """
    df_temp = df[[variavel_likert, variavel_grupo]].dropna().copy()
    df_temp['valor_numerico'] = df_temp[variavel_likert].map(mapa_numerico)
    
    medianas = df_temp.groupby(variavel_grupo)['valor_numerico'].agg(['median', 'mean', 'std', 'count'])
    medianas.columns = ['Mediana', 'Média', 'Desvio Padrão', 'N']
    medianas['Mediana_Texto'] = medianas['Mediana'].map(mapa_reverso)
    medianas = medianas.round(2)
    
    return medianas


print("✅ Funções auxiliares carregadas com sucesso!")

---

## 5. Análise Descritiva Geral

In [ ]:
# ============================================================================
# TABELA DE FREQUÊNCIAS PARA TODAS AS VARIÁVEIS LIKERT
# ============================================================================

print("=" * 90)
print("DISTRIBUIÇÃO DE FREQUÊNCIAS - ESCALA LIKERT (5 PONTOS)")
print("=" * 90)

# Criar tabela consolidada
tabela_frequencias = pd.DataFrame()

for var, descricao in variaveis_likert.items():
    freq = calcular_frequencias_likert(df, var)
    freq['Variável'] = nomes_curtos[var]
    tabela_frequencias = pd.concat([tabela_frequencias, freq], ignore_index=True)

# Pivotar para visualização
tabela_pivot = tabela_frequencias.pivot(index='Variável', columns='Nível', values='Percentual')
tabela_pivot = tabela_pivot[ordem_likert]  # Reordenar colunas

print("\nPercentual por nível de concordância (%):\n")
print(tabela_pivot.round(1))

In [ ]:
# ============================================================================
# MEDIDAS DE TENDÊNCIA CENTRAL
# ============================================================================

print("\n" + "=" * 90)
print("MEDIDAS DE TENDÊNCIA CENTRAL")
print("=" * 90)

medidas_centrais = []

for var, descricao in variaveis_likert.items():
    medidas = calcular_medidas_centrais(df, var)
    medidas['Variável'] = nomes_curtos[var]
    medidas['Descrição'] = descricao
    medidas_centrais.append(medidas)

df_medidas = pd.DataFrame(medidas_centrais)
df_medidas = df_medidas[['Variável', 'media', 'mediana', 'mediana_texto', 'moda_texto', 'desvio_padrao', 'n']]
df_medidas.columns = ['Variável', 'Média', 'Mediana', 'Mediana (texto)', 'Moda', 'Desvio Padrão', 'N']

print("\n")
print(df_medidas.to_string(index=False))

print("\n📊 Interpretação da escala numérica:")
print("   1 = Discordo totalmente | 2 = Discordo | 3 = Neutro | 4 = Concordo | 5 = Concordo totalmente")

In [ ]:
# ============================================================================
# CONCORDÂNCIA AGRUPADA (3 CATEGORIAS)
# ============================================================================

print("\n" + "=" * 90)
print("CONCORDÂNCIA AGRUPADA (Concordância / Neutro / Discordância)")
print("=" * 90)

tabela_agrupada = pd.DataFrame()

for var, descricao in variaveis_likert.items():
    freq_agrup = calcular_concordancia_agrupada(df, var)
    freq_agrup['Variável'] = nomes_curtos[var]
    tabela_agrupada = pd.concat([tabela_agrupada, freq_agrup], ignore_index=True)

# Pivotar para visualização
tabela_agrup_pivot = tabela_agrupada.pivot(index='Variável', columns='Categoria', values='Percentual')
tabela_agrup_pivot = tabela_agrup_pivot[ordem_agrupado]  # Reordenar colunas

print("\nPercentual por categoria agrupada (%):\n")
print(tabela_agrup_pivot.round(1))

---

## 6. Visualizações Gerais

In [ ]:
# ============================================================================
# GRÁFICO DE BARRAS DIVERGENTES (LIKERT PLOT)
# ============================================================================

def criar_likert_plot(tabela_pivot, titulo, arquivo_saida):
    """
    Cria gráfico de barras divergentes para visualização de escala Likert.
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Preparar dados
    variaveis = tabela_pivot.index.tolist()
    n_vars = len(variaveis)
    
    # Calcular posições
    # Negativos à esquerda do centro, positivos à direita
    discordo_total = tabela_pivot['Discordo totalmente'].values
    discordo = tabela_pivot['Discordo'].values
    neutro = tabela_pivot['Neutro'].values
    concordo = tabela_pivot['Concordo'].values
    concordo_total = tabela_pivot['Concordo totalmente'].values
    
    # Posições y
    y_pos = np.arange(n_vars)
    
    # Plotar barras (do centro para fora)
    # Lado esquerdo (negativo)
    ax.barh(y_pos, -discordo_total, left=-(discordo + neutro/2), 
            color=cores_likert['Discordo totalmente'], label='Discordo totalmente', height=0.6)
    ax.barh(y_pos, -discordo, left=-neutro/2, 
            color=cores_likert['Discordo'], label='Discordo', height=0.6)
    
    # Centro (neutro - dividido)
    ax.barh(y_pos, -neutro/2, left=0, 
            color=cores_likert['Neutro'], height=0.6)
    ax.barh(y_pos, neutro/2, left=0, 
            color=cores_likert['Neutro'], label='Neutro', height=0.6)
    
    # Lado direito (positivo)
    ax.barh(y_pos, concordo, left=neutro/2, 
            color=cores_likert['Concordo'], label='Concordo', height=0.6)
    ax.barh(y_pos, concordo_total, left=neutro/2 + concordo, 
            color=cores_likert['Concordo totalmente'], label='Concordo totalmente', height=0.6)
    
    # Configurações
    ax.set_yticks(y_pos)
    ax.set_yticklabels(variaveis)
    ax.set_xlabel('Percentual (%)')
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    ax.axvline(x=0, color='black', linewidth=0.5)
    
    # Legenda
    handles, labels = ax.get_legend_handles_labels()
    # Remover duplicatas mantendo a ordem
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='lower right', fontsize=9)
    
    # Limites do eixo x
    ax.set_xlim(-100, 100)
    
    plt.tight_layout()
    plt.savefig(arquivo_saida, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nGráfico salvo como: {arquivo_saida}")


# Criar gráfico
criar_likert_plot(tabela_pivot, 
                  'Grau de Concordância com Afirmações sobre IA', 
                  'grafico_likert_geral_etapa4.png')

In [ ]:
# ============================================================================
# GRÁFICO DE BARRAS EMPILHADAS (100%)
# ============================================================================

fig, ax = plt.subplots(figsize=(12, 6))

# Preparar dados
variaveis = list(nomes_curtos.values())
dados_plot = tabela_pivot.reindex([nomes_curtos[v] for v in variaveis_likert.keys()])

# Cores na ordem correta
cores_ordem = [cores_likert[nivel] for nivel in ordem_likert]

# Plotar barras empilhadas
dados_plot.plot(kind='barh', stacked=True, ax=ax, color=cores_ordem, edgecolor='white', width=0.7)

ax.set_xlabel('Percentual (%)')
ax.set_ylabel('')
ax.set_title('Distribuição das Respostas por Afirmação', fontsize=14, fontweight='bold')
ax.legend(title='Nível de Concordância', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
ax.set_xlim(0, 100)

# Adicionar linha de 50%
ax.axvline(x=50, color='black', linestyle='--', linewidth=0.8, alpha=0.5)

plt.tight_layout()
plt.savefig('grafico_likert_detalhado_etapa4.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_likert_detalhado_etapa4.png")

In [ ]:
# ============================================================================
# GRÁFICO DE CONCORDÂNCIA AGRUPADA
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Preparar dados
dados_agrup = tabela_agrup_pivot.reindex([nomes_curtos[v] for v in variaveis_likert.keys()])
cores_agrup = [cores_agrupado[cat] for cat in ordem_agrupado]

# Plotar
dados_agrup.plot(kind='barh', stacked=True, ax=ax, color=cores_agrup, edgecolor='white', width=0.7)

ax.set_xlabel('Percentual (%)')
ax.set_ylabel('')
ax.set_title('Concordância Agrupada por Afirmação', fontsize=14, fontweight='bold')
ax.legend(title='Categoria', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xlim(0, 100)

# Adicionar percentuais nas barras
for i, var in enumerate(dados_agrup.index):
    cumsum = 0
    for cat in ordem_agrupado:
        valor = dados_agrup.loc[var, cat]
        if valor > 5:  # Só mostrar se > 5%
            ax.text(cumsum + valor/2, i, f'{valor:.0f}%', 
                    ha='center', va='center', fontsize=9, fontweight='bold', color='white')
        cumsum += valor

plt.tight_layout()
plt.savefig('grafico_likert_agrupado_etapa4.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_likert_agrupado_etapa4.png")

---

## 7. Análise Estratificada por Vínculo Institucional

In [ ]:
# ============================================================================
# MEDIANAS POR VÍNCULO INSTITUCIONAL
# ============================================================================

print("=" * 100)
print("MEDIANAS POR VÍNCULO INSTITUCIONAL")
print("=" * 100)

for var, descricao in variaveis_likert.items():
    print(f"\n📌 {nomes_curtos[var]}")
    print(f"   \"{descricao}\"")
    print("-" * 80)
    medianas = calcular_medianas_por_grupo(df, var)
    print(medianas.to_string())

In [ ]:
# ============================================================================
# TESTE DE KRUSKAL-WALLIS (COMPARAÇÃO ENTRE VÍNCULOS)
# ============================================================================

print("\n" + "=" * 100)
print("TESTE DE KRUSKAL-WALLIS: Comparação entre Vínculos Institucionais")
print("=" * 100)
print(f"\nH₀: Não há diferença nas distribuições entre os grupos")
print(f"H₁: Há diferença nas distribuições entre os grupos")
print(f"Nível de significância (α): {ALPHA}")
print("-" * 100)

resultados_kruskal = []

for var, descricao in variaveis_likert.items():
    resultado = teste_kruskal_wallis(df, var)
    resultado['descricao'] = nomes_curtos[var]
    resultados_kruskal.append(resultado)
    
    status = "✓ Significativo" if resultado['significativo'] else "✗ Não significativo"
    p_fmt = "< 0,001" if resultado['p_valor'] < 0.001 else f"= {resultado['p_valor']:.4f}"
    
    print(f"\n📌 {nomes_curtos[var]}")
    print(f"   H = {resultado['estatistica_H']}, p {p_fmt}")
    print(f"   η² = {resultado['eta_squared']} (efeito {resultado['efeito']})")
    print(f"   Resultado: {status}")

In [ ]:
# Criar DataFrame com resultados dos testes
df_kruskal = pd.DataFrame(resultados_kruskal)
df_kruskal['p_valor_fmt'] = df_kruskal['p_valor'].apply(lambda x: '< 0,001' if x < 0.001 else f'{x:.4f}')
df_kruskal['resultado'] = df_kruskal['significativo'].apply(lambda x: '✓ Significativo' if x else '✗ Não significativo')

df_kruskal_display = df_kruskal[['descricao', 'estatistica_H', 'p_valor_fmt', 'eta_squared', 'efeito', 'resultado']]
df_kruskal_display.columns = ['Afirmação', 'H', 'p-valor', 'η²', 'Efeito', 'Resultado']

print("\n" + "=" * 100)
print("RESUMO DOS TESTES DE KRUSKAL-WALLIS")
print("=" * 100)
print(df_kruskal_display.to_string(index=False))

In [ ]:
# ============================================================================
# GRÁFICO COMPARATIVO POR VÍNCULO
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, (var, descricao) in enumerate(variaveis_likert.items()):
    ax = axes[idx]
    
    # Calcular percentuais por vínculo
    tab_vinculo = pd.crosstab(df['Vinculo_Padronizado'], df[var], normalize='index') * 100
    tab_vinculo = tab_vinculo[[c for c in ordem_likert if c in tab_vinculo.columns]]
    
    # Plotar
    cores_plot = [cores_likert[c] for c in tab_vinculo.columns]
    tab_vinculo.plot(kind='barh', stacked=True, ax=ax, color=cores_plot, edgecolor='white', width=0.7)
    
    ax.set_title(nomes_curtos[var], fontsize=11, fontweight='bold')
    ax.set_xlabel('Percentual (%)')
    ax.set_ylabel('')
    ax.set_xlim(0, 100)
    ax.legend().set_visible(False)

# Remover subplot extra
axes[5].axis('off')

# Legenda única
handles = [plt.Rectangle((0,0),1,1, facecolor=cores_likert[nivel]) for nivel in ordem_likert]
fig.legend(handles, ordem_likert, loc='lower right', bbox_to_anchor=(0.98, 0.12), 
           title='Nível de Concordância', fontsize=9)

plt.suptitle('Distribuição das Respostas por Vínculo Institucional', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_likert_vinculo_etapa4.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_likert_vinculo_etapa4.png")

In [ ]:
# ============================================================================
# GRÁFICO DE MÉDIAS POR VÍNCULO (COM INTERVALO DE CONFIANÇA)
# ============================================================================

fig, ax = plt.subplots(figsize=(12, 6))

# Preparar dados
dados_medias = []
for var, descricao in variaveis_likert.items():
    df_temp = df[[var, 'Vinculo_Padronizado']].dropna().copy()
    df_temp['valor'] = df_temp[var].map(mapa_numerico)
    
    for vinculo in df['Vinculo_Padronizado'].dropna().unique():
        valores = df_temp[df_temp['Vinculo_Padronizado'] == vinculo]['valor']
        dados_medias.append({
            'Variável': nomes_curtos[var],
            'Vínculo': vinculo,
            'Média': valores.mean(),
            'Erro': valores.std() / np.sqrt(len(valores)) * 1.96  # IC 95%
        })

df_medias = pd.DataFrame(dados_medias)

# Plotar
vinculos = df_medias['Vínculo'].unique()
variaveis_plot = list(nomes_curtos.values())
x = np.arange(len(variaveis_plot))
width = 0.15

cores_vinculo = sns.color_palette("Set2", n_colors=len(vinculos))

for i, vinculo in enumerate(vinculos):
    dados_vinculo = df_medias[df_medias['Vínculo'] == vinculo]
    dados_vinculo = dados_vinculo.set_index('Variável').reindex(variaveis_plot)
    
    ax.bar(x + i*width, dados_vinculo['Média'], width, 
           label=vinculo, color=cores_vinculo[i], 
           yerr=dados_vinculo['Erro'], capsize=2)

ax.set_ylabel('Média (1-5)')
ax.set_title('Média de Concordância por Vínculo Institucional', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * (len(vinculos)-1) / 2)
ax.set_xticklabels(variaveis_plot, rotation=15, ha='right')
ax.legend(title='Vínculo', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_ylim(1, 5)
ax.axhline(y=3, color='gray', linestyle='--', linewidth=0.8, alpha=0.5, label='Neutro')

# Adicionar escala de referência
ax.set_yticks([1, 2, 3, 4, 5])
ax.set_yticklabels(['1\n(Discordo\ntotalmente)', '2\n(Discordo)', '3\n(Neutro)', '4\n(Concordo)', '5\n(Concordo\ntotalmente)'])

plt.tight_layout()
plt.savefig('grafico_likert_medias_etapa4.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_likert_medias_etapa4.png")

---

## 8. Resumo Executivo

In [ ]:
print("=" * 80)
print("RESUMO EXECUTIVO - ETAPA 4")
print("=" * 80)

print(f"""
📊 PRINCIPAIS ACHADOS

1. CONCORDÂNCIA GERAL COM AS AFIRMAÇÕES
""")

for var, descricao in variaveis_likert.items():
    concordancia = tabela_agrup_pivot.loc[nomes_curtos[var], 'Concordância']
    medidas = calcular_medidas_centrais(df, var)
    print(f"   • {nomes_curtos[var]}: {concordancia:.1f}% de concordância (média: {medidas['media']})")

print(f"""
2. AFIRMAÇÃO COM MAIOR CONCORDÂNCIA
   • {tabela_agrup_pivot['Concordância'].idxmax()}: {tabela_agrup_pivot['Concordância'].max():.1f}%

3. AFIRMAÇÃO COM MAIOR DISCORDÂNCIA
   • {tabela_agrup_pivot['Discordância'].idxmax()}: {tabela_agrup_pivot['Discordância'].max():.1f}%

4. DIFERENÇAS ENTRE VÍNCULOS (Teste de Kruskal-Wallis)
""")

for resultado in resultados_kruskal:
    status = "✓" if resultado['significativo'] else "✗"
    print(f"   {status} {resultado['descricao']}: H = {resultado['estatistica_H']}, p {'< 0,001' if resultado['p_valor'] < 0.001 else f'= {resultado["p_valor"]:.4f}'} (efeito {resultado['efeito']})")

# Contar significativos
n_significativos = sum(1 for r in resultados_kruskal if r['significativo'])
print(f"""
📌 CONCLUSÃO:
   {n_significativos} de 5 afirmações apresentam diferenças significativas entre vínculos.
""")

---

## 9. Salvar Resultados

In [ ]:
# Salvar todas as tabelas em um arquivo Excel com múltiplas abas
with pd.ExcelWriter(ARQUIVO_SAIDA_EXCEL, engine='openpyxl') as writer:
    # Frequências detalhadas
    tabela_pivot.to_excel(writer, sheet_name='Freq_Likert_5pts')
    
    # Concordância agrupada
    tabela_agrup_pivot.to_excel(writer, sheet_name='Concordancia_Agrupada')
    
    # Medidas centrais
    df_medidas.to_excel(writer, sheet_name='Medidas_Centrais', index=False)
    
    # Medianas por vínculo (para cada variável)
    for var, descricao in variaveis_likert.items():
        medianas = calcular_medianas_por_grupo(df, var)
        nome_aba = f'Med_{nomes_curtos[var][:20]}'  # Limitar nome da aba
        medianas.to_excel(writer, sheet_name=nome_aba)
    
    # Resultados Kruskal-Wallis
    df_kruskal_display.to_excel(writer, sheet_name='Testes_KruskalWallis', index=False)

print(f"✅ Tabelas salvas em: {ARQUIVO_SAIDA_EXCEL}")

In [ ]:
# Gerar relatório em Markdown
relatorio = f"""# RELATÓRIO DE ANÁLISE DAS VARIÁVEIS DE ESCALA LIKERT
## Etapa 4 — Consulta à Comunidade UFPE sobre o Uso de IA

**Data de execução:** {datetime.now().strftime("%d/%m/%Y às %H:%M")}

**Nível de significância (α):** {ALPHA}

---

## 1. AFIRMAÇÕES AVALIADAS

| # | Variável | Afirmação |
|---|----------|----------|
"""

for i, (var, descricao) in enumerate(variaveis_likert.items(), 1):
    relatorio += f"| {i} | {nomes_curtos[var]} | {descricao} |\n"

relatorio += f"""
---

## 2. DISTRIBUIÇÃO DE FREQUÊNCIAS (5 PONTOS)

| Afirmação | Discordo totalmente | Discordo | Neutro | Concordo | Concordo totalmente |
|-----------|---------------------|----------|--------|----------|--------------------|
"""

for var in variaveis_likert.keys():
    nome = nomes_curtos[var]
    linha = f"| {nome} |"
    for nivel in ordem_likert:
        valor = tabela_pivot.loc[nome, nivel]
        linha += f" {valor:.1f}% |"
    relatorio += linha + "\n"

relatorio += f"""
---

## 3. CONCORDÂNCIA AGRUPADA

| Afirmação | Discordância | Neutro | Concordância |
|-----------|--------------|--------|-------------|
"""

for var in variaveis_likert.keys():
    nome = nomes_curtos[var]
    disc = tabela_agrup_pivot.loc[nome, 'Discordância']
    neut = tabela_agrup_pivot.loc[nome, 'Neutro']
    conc = tabela_agrup_pivot.loc[nome, 'Concordância']
    relatorio += f"| {nome} | {disc:.1f}% | {neut:.1f}% | {conc:.1f}% |\n"

relatorio += f"""
---

## 4. MEDIDAS DE TENDÊNCIA CENTRAL

| Afirmação | Média | Mediana | Moda | Desvio Padrão | N |
|-----------|-------|---------|------|---------------|---|
"""

for _, row in df_medidas.iterrows():
    relatorio += f"| {row['Variável']} | {row['Média']} | {row['Mediana (texto)']} | {row['Moda']} | {row['Desvio Padrão']} | {row['N']} |\n"

relatorio += f"""
**Escala:** 1 = Discordo totalmente | 2 = Discordo | 3 = Neutro | 4 = Concordo | 5 = Concordo totalmente

---

## 5. TESTE DE KRUSKAL-WALLIS (Comparação entre Vínculos)

**Hipóteses:**
- H₀: Não há diferença nas distribuições entre os grupos de vínculo
- H₁: Há diferença nas distribuições entre os grupos de vínculo

| Afirmação | H | p-valor | η² | Efeito | Resultado |
|-----------|---|---------|-----|--------|----------|
"""

for _, row in df_kruskal_display.iterrows():
    relatorio += f"| {row['Afirmação']} | {row['H']} | {row['p-valor']} | {row['η²']} | {row['Efeito']} | {row['Resultado']} |\n"

relatorio += f"""
---

## 6. PRINCIPAIS CONCLUSÕES

1. **Alta concordância geral:** Todas as 5 afirmações apresentaram níveis elevados de concordância (acima de {tabela_agrup_pivot['Concordância'].min():.1f}%).

2. **Maior concordância:** "{tabela_agrup_pivot['Concordância'].idxmax()}" ({tabela_agrup_pivot['Concordância'].max():.1f}% de concordância).

3. **Diferenças por vínculo:** {n_significativos} de 5 afirmações apresentam diferenças estatisticamente significativas entre os grupos de vínculo institucional.

4. **Consenso institucional:** A comunidade UFPE demonstra alto grau de alinhamento quanto à necessidade de diretrizes e debates sobre IA.

---

## 7. ARQUIVOS GERADOS

| Arquivo | Descrição |
|---------|----------|
| `{ARQUIVO_SAIDA_EXCEL}` | Tabelas de frequência e testes estatísticos |
| `grafico_likert_geral_etapa4.png` | Barras divergentes (visão geral) |
| `grafico_likert_detalhado_etapa4.png` | Barras empilhadas (100%) |
| `grafico_likert_agrupado_etapa4.png` | Concordância agrupada |
| `grafico_likert_vinculo_etapa4.png` | Comparação por vínculo |
| `grafico_likert_medias_etapa4.png` | Médias por vínculo com IC 95% |

---

*Relatório gerado automaticamente em {datetime.now().strftime("%d/%m/%Y às %H:%M")}*
"""

# Salvar relatório
with open(ARQUIVO_SAIDA_RELATORIO, "w", encoding='utf-8') as f:
    f.write(relatorio)

print(f"✅ Relatório salvo em: {ARQUIVO_SAIDA_RELATORIO}")

In [ ]:
print("\n" + "=" * 70)
print("ETAPA 4 CONCLUÍDA COM SUCESSO!")
print("=" * 70)
print(f"""
📁 ARQUIVOS GERADOS:

   📊 Dados:
      • {ARQUIVO_SAIDA_EXCEL}

   📈 Gráficos:
      • grafico_likert_geral_etapa4.png
      • grafico_likert_detalhado_etapa4.png
      • grafico_likert_agrupado_etapa4.png
      • grafico_likert_vinculo_etapa4.png
      • grafico_likert_medias_etapa4.png

   📝 Relatório:
      • {ARQUIVO_SAIDA_RELATORIO}

▶️  PRÓXIMA ETAPA: Etapa 5 - Análise das Variáveis de Múltipla Escolha
""")

---

## Resumo dos Arquivos

### Entrada
| Arquivo | Origem |
|---------|--------|
| `dados_limpos_etapa1.xlsx` | Etapa 1 |

### Saída
| Arquivo | Descrição |
|---------|----------|
| `likert_etapa4.xlsx` | Tabelas de frequência, medidas descritivas e testes (10 abas) |
| `grafico_likert_geral_etapa4.png` | Barras divergentes - visão geral |
| `grafico_likert_detalhado_etapa4.png` | Barras empilhadas 100% |
| `grafico_likert_agrupado_etapa4.png` | Concordância agrupada (3 categorias) |
| `grafico_likert_vinculo_etapa4.png` | Comparação entre vínculos |
| `grafico_likert_medias_etapa4.png` | Médias por vínculo com IC 95% |
| `relatorio_likert_etapa4.md` | Relatório completo com testes estatísticos |

---

**Próxima etapa:** Etapa 5 - Análise das Variáveis de Múltipla Escolha